In [1]:
# color
# white（255，255，255）non
# black（0，0，0）I
# red（255，0，0）D
# green（0，255，0）M
# blue（0，0，255）S

import sys
import pysam
import numpy as np
import os
from PIL import Image
import math

ModuleNotFoundError: No module named 'pysam'

In [2]:
#hg003的路径
bam_file_path = r'/mnt/hde/gao/lyx/hg002/bam/'

#chr_single_area_path = r'/Users/ksg/hg002/ngmlr/chr_single_area/'
#save_path = r'/Users/ksg/hg002/ngmlr/'
chr_single_area_path = r'/mnt/hde/gao/lyx/hg002/tools_merged_pics/chr_single_area/'
save_pics_path = r'/mnt/hde/gao/lyx/hg002/tools_merged_pics/'
ch = ['chr1', 'chr2', 'chr3', 'chr7', 'chr9', 'chr11', 'chr13', 'chr21', 'chr22']
#ch = ['chr19','chr22']

In [3]:
def get_bam_full_name(bam_file_path,chid):
    return bam_file_path + 'ngmlr_hg19_hg002_sorted_'+chid+'.subreads.bam'

In [4]:
def binarySearch (arr, l, r, x): #二分查找到目标碱基的index
    #depth_file数组，起始，终止，目标数子
    if r >= l: 
        mid = int(l + (r - l)/2)
        if int(arr[mid].strip().split('\t')[1]) == x: 
            return mid 
        elif int(arr[mid].strip().split('\t')[1]) > x: 
            return binarySearch(arr, l, mid-1, x) 
        else: 
            return binarySearch(arr, mid+1, r, x)   
    else: 
        return -1

def read_cigartuples_remove_redundancy(read_cigartuples):
    read_cigartuples_1 = [read_cigartuples[0]]
    for i in range(1,len(read_cigartuples)):
        if read_cigartuples[i][0] == read_cigartuples_1[-1][0]:
            read_cigartuples_1[-1][1] += read_cigartuples[i][1]
        else:
            read_cigartuples_1.append(read_cigartuples[i])
    return np.array(read_cigartuples_1)


def get_area_read_seq_inf(open_bam_file,chid,start,stop):

    read_temp,cigartuples_temp = [],[]
    for read in open_bam_file.fetch(contig=chid[3:],start=max(0,start-1000),stop=stop+1000):

        
        if read.is_secondary == False:
            read_cigartuples = np.array(read.cigartuples)
            maped_type = read_cigartuples[:,0]
            if (3 not in maped_type[1:-1]) and (4 not in maped_type[1:-1]) and (5 not in maped_type[1:-1]) and (6 not in maped_type[1:-1]) and (7 not in maped_type[1:-1]) and (8 not in maped_type[1:-1]):
                if (read_cigartuples[0][0] in [0,2,4]) and (read_cigartuples[-1][0] in [0,2,4]):
                    read_name = read.query_name
                    if int(read_cigartuples[0][0]) != 4:
                        read_start = read.reference_start + 1
                    else:
                        read_start = read.reference_start - read_cigartuples[0][1] + 1
                    if int(read_cigartuples[-1][0]) != 4:
                        read_end = read.reference_end
                    else:
                        read_end = read.reference_end + read_cigartuples[-1][1]

            if (read_end<start) or (read_start>stop):
                continue
            else:
                read_temp.append([read_name,read_start,read_end])
                del_mini_area_index = []
                for j in range(len(read_cigartuples)):
                    if (read_cigartuples[j][1] <= 3) and (read_cigartuples[j][0] == 1):
                        del_mini_area_index.append(j)
                read_cigartuples = np.delete(read_cigartuples,del_mini_area_index,axis=0)
                for k in range(len(read_cigartuples)): 
                    if read_cigartuples[k][1] <= 3:
                        read_cigartuples[k][0] = 0
                        
                read_cigartuples = read_cigartuples_remove_redundancy(read_cigartuples)
                
                for i in range(1,len(read_cigartuples)-1):
                    if read_cigartuples[i][0] == 1:
                        if read_cigartuples[i-1][1] >= read_cigartuples[i+1][1]:
                            read_cigartuples[i-1][1] -= 1
                        else:
                            read_cigartuples[i+1][1] -= 1
                        read_cigartuples[i][1] = 1
                del_model_index = []
                for u in range(len(read_cigartuples)):
                    if read_cigartuples[u][1] == 0:
                        del_model_index.append(u)
                read_cigartuples = np.delete(read_cigartuples,del_model_index,axis=0)
                read_cigartuples = read_cigartuples_remove_redundancy(read_cigartuples)

                cigartuples_temp.append(read_cigartuples)
    return read_temp,cigartuples_temp

def cigar_num_transform(num):
    if num == 0:    #M
        return [1]
    elif num == 1:  #I
        return [2]
    elif num == 2:  #D
        return [3]
    elif num == 4:  #S
        return [4]

def rearrange_read(read_temp,cigartuples_temp,start,stop):

    zero_threshold = np.zeros(shape=(len(read_temp),stop-start+1)) #生成全0画布
    temp = []
    for i in range(len(read_temp)):
        read_start = read_temp[i][1]
        read_stop = read_temp[i][2]

        if read_start >= start:
            for j3 in range(len(cigartuples_temp[i])):
                zero_threshold[i,(read_start-start):min((stop-start+1),(read_start+cigartuples_temp[i][j3][1]-start))] = np.array((min(stop-start+1,read_start+cigartuples_temp[i][j3][1]-start)-(read_start-start))*cigar_num_transform(cigartuples_temp[i][j3][0]))
                read_start += cigartuples_temp[i][j3][1]
                

        elif read_start < start:

            for k2 in range(len(cigartuples_temp[i])):
                if (read_start + cigartuples_temp[i][k2][1]) >= start:
                    read_start_index = read_start
                    cigartuples_index = k2
                    break
                else:
                    read_start += cigartuples_temp[i][k2][1]

            for k4 in range(cigartuples_index,len(cigartuples_temp[i])):
                zero_threshold[i,max(0,read_start_index-start):min(read_start_index+cigartuples_temp[i][k4][1]-start,stop-start+1)] = np.array((min(read_start_index+cigartuples_temp[i][k4][1]-start,stop-start+1)-max(0,read_start_index-start))*cigar_num_transform(cigartuples_temp[i][k4][0]))
                read_start_index += cigartuples_temp[i][k4][1]     
    return zero_threshold

def get_array_nonzero_area(array):

    temp = []
    output_temp = []
    for i in range(len(array)):
        if array[i] != 0:
            temp.append(i)
    output_temp.append(temp[0])
    output_temp.append(temp[-1])
    return output_temp

def remove_redundancy_blank_space(zero_threshold):

    nonzero_area_temp = []
    max_read_index_temp = []
    none_max_read_index_temp = []
    already_moved_read_index = []
    height = zero_threshold.shape[0]
    length = zero_threshold.shape[1]

    for i in range(len(zero_threshold)):
        output_temp = get_array_nonzero_area(zero_threshold[i])
        nonzero_area_temp.append(output_temp) 

    for j in range(len(nonzero_area_temp)):
        if (nonzero_area_temp[j][-1] - nonzero_area_temp[j][0] + 1) == length:

            continue
        else:
            none_max_read_index_temp.append(j)
    
    if len(none_max_read_index_temp) < 2:
        return zero_threshold
    else:
        for k in range(len(none_max_read_index_temp)-1):
            for l in range(1,len(none_max_read_index_temp)):
                target_read_index = none_max_read_index_temp[k]
                need_move_read_index = none_max_read_index_temp[l]
                if (target_read_index not in already_moved_read_index) and (need_move_read_index not in already_moved_read_index):
                    left = nonzero_area_temp[need_move_read_index][0]
                    right = nonzero_area_temp[need_move_read_index][1]
                    if sum((zero_threshold[need_move_read_index,left:right+1] + zero_threshold[target_read_index,left:right+1]) == zero_threshold[need_move_read_index,left:right+1]) == (right - left + 1):
                        already_moved_read_index.append(need_move_read_index)
                        zero_threshold[target_read_index] += zero_threshold[need_move_read_index]
        return np.delete(zero_threshold,already_moved_read_index,axis=0)

def get_rgb(save_path,zero_threshold,start,stop,chid,pic_range):

    im = Image.new("RGB",((stop-start+1),len(zero_threshold)))
    for i in range(len(zero_threshold)):
        for j in range(stop-start+1):
            if zero_threshold[i][j] == 0:
                im.putpixel((j,i),(255,255,255))
            elif zero_threshold[i][j] == 1:
                im.putpixel((j,i),(0,255,0))
            elif zero_threshold[i][j] == 2:
                im.putpixel((j,i),(0,0,0))
            elif zero_threshold[i][j] == 3:
                im.putpixel((j,i),(255,0,0))
            elif zero_threshold[i][j] == 4:
                im.putpixel((j,i),(0,0,255))
    out = im.resize((100,100),Image.ANTIALIAS)

    out.save(save_path+chid+'_'+pic_range+'.png')

def save_pics(open_bam_file,save_path,bam_file_path,chid,start,stop,pic_range):

    read_temp,cigartuples_temp = get_area_read_seq_inf(open_bam_file,chid,start,stop)
    zero_threshold_1 = rearrange_read(read_temp,cigartuples_temp,start,stop)

    zero_threshold = remove_redundancy_blank_space(zero_threshold_1)

    if zero_threshold.shape[0] < 100:
        zeros = np.zeros(shape=(100-zero_threshold.shape[0],zero_threshold.shape[1]))
        c = np.vstack((zero_threshold,zeros))

        vv = get_rgb(save_path,c,start,stop,chid,pic_range)
    else:
        vv = get_rgb(save_path,zero_threshold,start,stop,chid,pic_range)

In [ ]:
def main(chr_single_area_path,save_pics_path,bam_file_path,chid):
    open_single_del_area_file = open(chr_single_area_path+chid+'_single_del_area.txt')
    single_del_area_file = open_single_del_area_file.readlines()
    single_del_area_file_len = len(single_del_area_file)
    open_single_del_area_file.close()
    
    bam_full_name = get_bam_full_name(bam_file_path,chid)
    open_bam_file = pysam.AlignmentFile(bam_full_name,'rb')
        
    for i in single_del_area_file:
        ccc = save_pics(open_bam_file,save_pics_path+chid+'/',bam_file_path,chid,int(i.strip().split('\t')[0]),int(i.strip().split('\t')[1]),i.strip().split('\t')[0]+'-'+i.strip().split('\t')[0])
    
    open_bam_file.close()

for chid in ch:
    a = main(chr_single_area_path,save_pics_path,bam_file_path,chid)
    print(chid+' ... done')